In [37]:
class Fuzzy:
    def __init__(self):  # Perbaikan _init_ menjadi __init__
        self.max = 0
        self.min = 0

    def down(self, x):
        return max(0, (self.max - x) / (self.max - self.min))

    def up(self, x):
        return max(0, (x - self.min) / (self.max - self.min))

    def rev_down(self, value):
        return self.max - value * (self.max - self.min)

    def rev_up(self, value):
        return value * (self.max - self.min) + self.min

In [38]:
class Demand(Fuzzy):
    def __init__(self):  # Perbaikan _init_ menjadi __init__
        self.min = 1000
        self.max = 5000

    def turun(self, x):
        if x < self.min:
            return 1
        if x > self.max:
            return 0
        return self.down(x)

    def tetap(self, x):
        mid = (self.min + self.max) / 2
        if x < self.min or x > self.max:
            return 0
        if x == mid:
            return 1
        return 1 - abs(x - mid) / (mid - self.min)

    def naik(self, x):
        if x < self.min:
            return 0
        if x > self.max:
            return 1
        return self.up(x)

In [39]:
class Stock(Fuzzy):
    def __init__(self):  # Perbaikan _init_ menjadi __init__
        self.min = 100
        self.max = 1000

    def sedikit(self, x):
        if x < self.min:
            return 1
        if x > self.max:
            return 0
        return self.down(x)

    def sedang(self, x):
        mid = (self.min + self.max) / 2
        if x < self.min or x > self.max:
            return 0
        if x == mid:
            return 1
        return 1 - abs(x - mid) / (mid - self.min)

    def banyak(self, x):
        if x < self.min:
            return 0
        if x > self.max:
            return 1
        return self.up(x)

In [40]:
class Production(Fuzzy):
    def __init__(self):  # Perbaikan _init_ menjadi __init__
        self.min = 2000
        self.max = 8000

    def berkurang(self, fuzzy_value):
        return self.rev_down(fuzzy_value)

    def bertambah(self, fuzzy_value):
        return self.rev_up(fuzzy_value)

In [41]:
# Rule map sesuai dengan aturan
rule_map = {
    ("turun", "sedikit"): "bertambah",
    ("turun", "sedang"): "berkurang",
    ("turun", "banyak"): "berkurang",
    ("tetap", "sedikit"): "bertambah",
    ("tetap", "sedang"): "berkurang",
    ("tetap", "banyak"): "berkurang",
    ("naik", "sedikit"): "bertambah",
    ("naik", "sedang"): "bertambah",
    ("naik", "banyak"): "berkurang",
}

In [42]:
def fuzzy_production_inference(x_demand, x_stock):
    fuzzy_real = 0.0  # Inisialisasi dengan float
    fuzzy = 0.0  # Inisialisasi dengan float

    # Buat satu instance dari masing-masing kelas
    demand = Demand()
    stock = Stock()
    prod = Production()

    for k, v in rule_map.items():
        # Hitung nilai fuzzy untuk demand dan stock
        demand_value = getattr(demand, k[0])(x_demand)
        stock_value = getattr(stock, k[1])(x_stock)

        # Agregasi dengan metode minimum
        prod_value = min(demand_value, stock_value)

        # Defuzzifikasi
        x_prod = getattr(prod, v)(prod_value)

        # Hitung centroid
        fuzzy_real += prod_value * x_prod
        fuzzy += prod_value

    # Hitung hasil akhir
    result = int(fuzzy_real / fuzzy) if fuzzy != 0 else 0
    print(f"\nHasil Defuzzifikasi (Centroid): {result}")
    return result

In [49]:
# Contoh penggunaan
x_demand = 3500
x_stock = 300
result = fuzzy_production_inference(x_demand, x_stock)
print(f"\nProduksi yang disarankan: {result}")


Hasil Defuzzifikasi (Centroid): 5734

Produksi yang disarankan: 5734
